In [1]:
import os
import pprint
from pathlib import Path

import tensorflow as tf
import json
import yaml
import pandas as pd
from tqdm import tqdm

from src.data.monuseg.tf_data import get_dataset
from src.models.monuseg.train_model import eval, loss
from src.models.monuseg.models import get_model

os.environ["CUDA_VISIBLE_DEVICES"] = "3"
pp = pprint.PrettyPrinter(indent=4)

In [2]:
path_indices = "/home/valentin/python_wkspce/2d_bispectrum_cnn/data/indices/monuseg.json"
config = "/home/valentin/python_wkspce/2d_bispectrum_cnn/src/models/monuseg/configs/bispect_nh1.yaml"
# config = "/home/valentin/python_wkspce/2d_bispectrum_cnn/src/models/monuseg/configs/unet_default.yaml"
with open(path_indices, "r") as f:
    indices_list = json.load(f)


with open(config, 'r') as f:
    params = yaml.safe_load(f)

In [3]:
ds_test = get_dataset(id_list=indices_list[0]["test"], instance=True)
ds_test = ds_test.cache().batch(1)

In [4]:
params["n_harmonics"] = 8
params

{'model_name': 'BispectUnet',
 'rotation': True,
 'n_harmonics': 8,
 'cosine_decay': False,
 'n_train': -1,
 'n_feature_maps': [8, 16, 32],
 'patch_size': [60, 60],
 'batch_size': 16}

In [5]:
model = get_model(model_name=params["model_name"],
                  output_channels=3,
                  loss=loss,
                  n_harmonics=params["n_harmonics"],
                  cosine_decay=params["cosine_decay"],
                  last_activation="softmax",
                  run_eagerly=False)


In [6]:
model.load_weights('/home/valentin/python_wkspce/2d_bispectrum_cnn/models/MoNuSeg/BispectUnet__rotation_True__nh_8__n_train_-1__psize_60x60__20211202-173734/weights/split_0/final')
# model.load_weights('/home/valentin/python_wkspce/2d_bispectrum_cnn/models/MoNuSeg/BispectUnet__rotation_True__nh_8__n_train_-1__psize_60x60__20211202-173734/weights/split_0/final')
# model = tf.keras.models.load_model(
#     '/home/valentin/python_wkspce/2d_bispectrum_cnn/models/test',
#     compile=False)
# model.compile(
#     loss=[loss],
#     optimizer=tf.keras.optimizers.Adam(1e-3),
#     run_eagerly=False,
# )
# eval


In [7]:
# scores_test = eval(ds=ds_test,
#                    model=model,
#                    cropper=tf.keras.layers.Cropping2D(cropping=(20, 20)))
                   
# pp.pprint(scores_test)

In [8]:
model_path = Path(
    "/home/valentin/python_wkspce/2d_bispectrum_cnn/models/MoNuSeg/"
    "BispectUnet__rotation_True__nh_8__n_train_-1__psize_60x60__20211202-173734"
    "/weights/")

# model_path = Path(
#     "/home/valentin/python_wkspce/2d_bispectrum_cnn/models/MoNuSeg/"
#     "Unet__rotation_True__nh_0__n_train_-1__psize_60x60__20211202-132039"
#     "/weights/")


In [9]:
cropper = tf.keras.layers.Cropping2D(cropping=(20, 20))
scores = pd.DataFrame()
for split in tqdm(range(10)):
    model.load_weights(model_path / f"split_{split}/final")
    ds_test = get_dataset(id_list=indices_list[split]["test"],
                          instance=True).batch(1)
    scores = scores.append(
        {
            "split": split,
            **eval(ds=ds_test, model=model, cropper=cropper),
        },
        ignore_index=True,
    )


100%|██████████| 10/10 [26:34<00:00, 159.49s/it]


In [10]:
scores

,aij,fscore,precision,recall,split
0,0.409412,0.745572,0.760966,0.749922,0.0
1,0.359154,0.743382,0.747770,0.752788,1.0
2,0.327390,0.691379,0.670249,0.733313,2.0
3,0.330561,0.699532,0.706942,0.699573,3.0
4,0.347168,0.741705,0.691470,0.809801,4.0
5,0.363134,0.710382,0.712970,0.729414,5.0
6,0.369630,0.761186,0.784844,0.749924,6.0
7,0.277081,0.701572,0.706412,0.706955,7.0
8,0.329653,0.711060,0.655524,0.786719,8.0
9,0.342614,0.651132,0.566521,0.781921,9.0


In [ ]:
scores = pd.read_csv("scores_nh8_test.csv")

In [11]:
scores.mean()

aij          0.345580
fscore       0.715690
precision    0.700367
recall       0.750033
split        4.500000
dtype: float64

In [ ]:
scores.std()

In [ ]:
scores.to_csv("scores_nh8_test.csv")